In [ ]:
import os
import mne
import pandas as pd

# Directorio donde están los archivos .set y .csv
set_dir = r"C:\Users\Usuario\Documents\Datos EEG\PWI_Datos_Procesados_pipeline_MALO\PWI_Set_4000_2"

# Diccionario para almacenar las épocas por condición
Epocas = {"con": {}, "rel": {}, "unrel": {}}

# Diccionario para almacenar marcas y eventos por condición
Marcas_Eventos = {"con": {}, "rel": {}, "unrel": {}}

# Condiciones y sus correspondientes sufijos de archivo
condiciones = {
    "con": "PWI_CON",
    "rel": "PWI_INC_REL",
    "unrel": "PWI_INC_UNREL"
}

# Sujetos a excluir
sujetos_excluir = {11, 30}

# Recorre desde S1 hasta S40 para cada condición
for i in range(1, 41):
    sujeto_key = f"Sujeto_{i}"
    
    for condicion, sufijo in condiciones.items():
        if i in sujetos_excluir:
            Epocas[condicion][sujeto_key] = None
            Marcas_Eventos[condicion][sujeto_key] = None
            print(f"{sujeto_key} excluido de la carga en '{condicion}', asignado como None.")
            continue

        # Construir nombre de archivo .set
        file_name_set = f"S{i}_{sufijo}.set"
        file_path_set = os.path.join(set_dir, file_name_set)

        # Construir nombre de archivo .csv
        file_name_csv = f"S{i}_{sufijo}_Marcas_Eventos.csv"
        file_path_csv = os.path.join(set_dir, file_name_csv)

        # Manejo de archivos .set
        try:
            epochs = mne.io.read_epochs_eeglab(file_path_set)
            Epocas[condicion][sujeto_key] = epochs
            print(f"{file_name_set} cargado exitosamente en '{condicion}'.")
        except FileNotFoundError:
            Epocas[condicion][sujeto_key] = None
            print(f"{file_name_set} no encontrado en '{condicion}'.")
        except Exception as e:
            Epocas[condicion][sujeto_key] = None
            print(f"Error al cargar {file_name_set} en '{condicion}': {e}")

        # Manejo de archivos .csv
        try:
            marcas_eventos_df = pd.read_csv(file_path_csv)
            Marcas_Eventos[condicion][sujeto_key] = marcas_eventos_df
            print(f"{file_name_csv} cargado exitosamente en '{condicion}'.")
        except FileNotFoundError:
            Marcas_Eventos[condicion][sujeto_key] = None
            print(f"{file_name_csv} no encontrado en '{condicion}'.")
        except Exception as e:
            Marcas_Eventos[condicion][sujeto_key] = None
            print(f"Error al cargar {file_name_csv} en '{condicion}': {e}")


In [3]:
# Inicializa el diccionario para almacenar las marcas y eventos con el tiempo relativo
Marcas_Eventos_Relativo = {condicion: {} for condicion in Marcas_Eventos}

# Recorre cada condición y cada sujeto en el diccionario Marcas_Eventos
for condicion, sujetos in Marcas_Eventos.items():
    for sujeto, df in sujetos.items():
        # Si no hay datos, guarda None para este sujeto
        if df is None:
            Marcas_Eventos_Relativo[condicion][sujeto] = None
            print(f"{sujeto} en {condicion} no tiene datos de marcas y eventos.")
            continue

        # Obtén las épocas correspondientes a este sujeto en Epocas
        epochs = Epocas[condicion].get(sujeto, None)
        if epochs is None:
            print(f"No se encontraron épocas para {sujeto} en {condicion}.")
            Marcas_Eventos_Relativo[condicion][sujeto] = None
            continue

        # Extrae los datos de las épocas
        epochs_data = epochs.get_data()  # Forma: (n_epocas, n_canales, n_muestras)
        n_muestras = epochs_data.shape[2]
        
        # La duración total de la época en muestras es la diferencia entre el primer y último tiempo
        duracion_epoca = n_muestras / epochs.info['sfreq']  # Duración en segundos

        # Crea una nueva columna de "Tiempo_Relativo"
        # Calculamos el tiempo relativo de cada marca
        df['Tiempo_Relativo'] = df['Latency'] / (duracion_epoca * 1000) * 100  # en porcentaje

        # Guarda el dataframe modificado en el nuevo diccionario
        Marcas_Eventos_Relativo[condicion][sujeto] = df
        print(f"Tiempo relativo calculado y agregado para {sujeto} en {condicion}.")

# Ahora el diccionario Marcas_Eventos_Relativo contiene los datos originales de marcas y eventos
# con la nueva columna 'Tiempo_Relativo' añadida.


Tiempo relativo calculado y agregado para Sujeto_1 en con.
Tiempo relativo calculado y agregado para Sujeto_2 en con.
Tiempo relativo calculado y agregado para Sujeto_3 en con.
Tiempo relativo calculado y agregado para Sujeto_4 en con.
Tiempo relativo calculado y agregado para Sujeto_5 en con.
Tiempo relativo calculado y agregado para Sujeto_6 en con.
Tiempo relativo calculado y agregado para Sujeto_7 en con.
Tiempo relativo calculado y agregado para Sujeto_8 en con.
Tiempo relativo calculado y agregado para Sujeto_9 en con.
Tiempo relativo calculado y agregado para Sujeto_10 en con.
Sujeto_11 en con no tiene datos de marcas y eventos.
Sujeto_12 en con no tiene datos de marcas y eventos.
Tiempo relativo calculado y agregado para Sujeto_13 en con.
Tiempo relativo calculado y agregado para Sujeto_14 en con.
Tiempo relativo calculado y agregado para Sujeto_15 en con.
Tiempo relativo calculado y agregado para Sujeto_16 en con.
Tiempo relativo calculado y agregado para Sujeto_17 en con.
Tie